In [1]:
%load_ext autoreload
%autoreload 2

from torch import empty, cat, arange
from torch.nn.functional import fold, unfold
from torch import nn

from model import *

In [4]:
# loading image data
noisy_imgs_1, noisy_imgs_2 = torch.load('../../data/train_data.pkl')
train_input, train_target = noisy_imgs_1.float()/255.0, noisy_imgs_2.float()/255.0

In [3]:
# take batch of size 1
input = train_input[0].view(1, 3, 32, 32)
target = train_target[0].view(1, 3, 32, 32)
input.shape

torch.Size([1, 3, 32, 32])

In [10]:
# create some tensors for simple tests
x = torch.randn((1, 3, 4, 4))

# y = torch.ones(x.shape)
y = torch.ones((1, 4, 3, 3))

x, y

(tensor([[[[-0.7781,  1.0103, -1.5367,  0.1919],
           [-0.2604,  0.1187, -0.9530, -0.0493],
           [ 0.2665,  0.1257,  2.2312,  0.8957],
           [ 0.3404, -0.2406, -0.5551, -0.6570]],
 
          [[-0.5045, -0.3335, -0.2546, -0.5980],
           [-1.1061, -0.1766, -0.1381, -0.5813],
           [ 0.1311, -0.5913,  1.4443, -1.0994],
           [ 0.1172,  0.9564,  0.1378, -0.1556]],
 
          [[ 0.7186, -0.8934, -0.3290, -0.8755],
           [ 0.5194,  0.2253, -0.0102,  0.5849],
           [ 1.7041, -1.7435, -1.2237, -0.8067],
           [ 0.0472, -1.6299,  0.9207,  0.9559]]]]),
 tensor([[[[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]],
 
          [[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]]]]))

### Testing our Conv2d Layer 

Forward pass

In [12]:
input.shape

torch.Size([1, 3, 32, 32])

In [69]:
# testing our conv2d layer
my_conv = Conv2d(3, 4, 2)
output = my_conv.forward(x)
output.shape

torch.Size([1, 4, 3, 3])

In [64]:
torch.testing.assert_allclose(my_conv.forward(input), torch.nn.functional.conv2d(input, my_conv.weight, my_conv.bias))

In [42]:

criterion = MSE()

loss = criterion.forward(output, my_conv.forward(target))
loss.requires_grad_()
l_grad = loss.backward()
l_grad.shape

AttributeError: 'NoneType' object has no attribute 'shape'

Backward pass

In [43]:
my_conv.input_x.shape[1]

12

In [44]:
output.shape

torch.Size([1, 4, 31, 31])

In [48]:
my_conv.weight.view(my_conv.output_channels, -1).shape

torch.Size([4, 12])

In [57]:
output.shape

torch.Size([1, 4, 31, 31])

In [54]:
grdoutput = (my_conv.weight.view(my_conv.output_channels, -1).t() @ output.view(1, my_conv.output_channels, -1))

In [59]:
input.shape

torch.Size([1, 3, 32, 32])

In [56]:
fold(grdoutput, output_size=input.shape[2:], kernel_size=my_conv.kernel_size, padding=my_conv.padding, stride=my_conv.stride).shape

torch.Size([1, 3, 32, 32])

In [77]:
my_conv.input_x.shape, my_conv.bias.shape

(torch.Size([1, 3, 4, 4]), torch.Size([4]))

In [71]:
my_conv.backward(output)

tensor([[[[-0.1133,  0.4086, -0.3195,  0.1317],
          [-0.1134,  0.1100, -0.2876,  0.1214],
          [ 0.4692, -0.6831,  1.4075, -0.0398],
          [ 0.0674, -0.6124,  0.2772,  0.0436]],

         [[-0.0764,  0.0615, -0.1131,  0.1226],
          [-0.5011,  0.0688,  0.0460,  0.0334],
          [-0.2054, -0.2830,  0.8174, -0.0631],
          [-0.0350,  0.0097,  0.9152,  0.2967]],

         [[ 0.2333, -0.1425, -0.0682,  0.0146],
          [-0.0450,  0.4051, -0.2960,  0.3481],
          [ 0.4077, -1.3182,  0.1713, -0.3866],
          [ 0.0984, -0.5252,  0.5618,  0.1257]]]])

### Testing our Upsampling (TransposeConv2d) layer

Forward pass

In [125]:
my_t_conv = Upsampling(3, 5, 2, stride=2)
upsampled = my_t_conv.forward(input)
upsampled.shape

torch.Size([1, 5, 64, 64])

In [128]:
torch.testing.assert_allclose(my_t_conv.forward(input), torch.nn.functional.conv_transpose2d(input, my_t_conv.weight, my_t_conv.bias, stride=2))

Backward pass

In [130]:
my_t_conv.backward(upsampled).shape

torch.Size([1, 3, 32, 32])

### Sequential model testing

In [144]:
model = Sequential([Conv2d(3,4,2), ReLU()])
forward_pass = model.forward(x)
backward_pass = model.backward(forward_pass)


tensor([[[[ 1.6125e-01,  3.4609e-01, -1.8182e-01,  4.0494e-01],
          [ 2.5950e-01, -1.7082e-02, -7.9798e-01,  1.7086e-01],
          [ 1.6125e-01,  1.8859e-01,  8.5036e-01,  2.9853e-01],
          [ 2.5950e-01, -2.4566e-02, -6.0386e-01, -5.5168e-01]],

         [[-1.7628e-01,  3.8483e-02,  6.5421e-01, -5.0083e-02],
          [-2.7424e-01,  5.6687e-04,  3.3949e-01,  2.9070e-01],
          [-1.7628e-01,  2.9196e-01,  8.8793e-01,  2.9944e-01],
          [-2.7424e-01,  1.5897e-01, -2.5462e-01,  1.3513e-01]],

         [[-1.1057e-01, -3.4472e-01, -2.6800e-01,  5.3072e-01],
          [-2.8320e-02,  1.1625e-01,  5.6445e-01,  6.7065e-01],
          [-1.1057e-01, -3.6104e-01, -8.0202e-02,  3.0841e-01],
          [-2.8320e-02,  2.7095e-01,  8.4798e-01,  3.6433e-01]]]])

### Other tests

In [5]:
conv.weight.view(out_channels, -1).size()

torch.Size([4, 12])

In [118]:
x.size()

torch.Size([1, 3, 4, 4])

In [112]:
unfolded.size()

torch.Size([1, 12, 9])

In [115]:

wxb.size()

torch.Size([1, 4, 9])

In [81]:
# relu = ReLU()
f=Sigmoid()

f.forward(x)

tensor([[0.5000, 0.7311, 0.5000],
        [0.8808, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.7311]])

In [82]:
f.backward(y)

tensor([[0.5000, 0.2689, 0.5000],
        [0.1192, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.2689]])